In [ ]:
WITH 
-- 新增：计算当日版本序号（核心改动1）
version_seq AS (
    SELECT 
        CONCAT('V', DATE_FORMAT(CURRENT_TIMESTAMP(), '%Y%m%d')) AS base_version,
        -- 统计当日已存在的版本数 + 1 = 本次序号
        COALESCE(MAX(CAST(SUBSTRING_INDEX(version, '-', -1) AS UNSIGNED)), 0) + 1 AS seq
    FROM fill_power_energy_year_predict  -- 关联目标表，统计已有版本
    WHERE version LIKE CONCAT('V', DATE_FORMAT(CURRENT_TIMESTAMP(), '%Y%m%d'), '-%')
),
-- 1. 2026年全年月度维度（直接生成1-12月）
year_month_2026 AS (
    SELECT 
        2026 AS `year`, 
        month AS `month`
    FROM (
        SELECT 1 AS month UNION ALL SELECT 2 UNION ALL SELECT 3 UNION ALL SELECT 4 UNION ALL SELECT 5 UNION ALL SELECT 6 
        UNION ALL SELECT 7 UNION ALL SELECT 8 UNION ALL SELECT 9 UNION ALL SELECT 10 UNION ALL SELECT 11 UNION ALL SELECT 12
    ) t
),
-- 2. 场站基础信息（取2026年的基础数据，去重）
station_info AS (
    SELECT
        station_code, 
        station_name, 
        station_type,
        src_region_name, 
        src_province_name
    FROM ads.ads_stnops_energy_monthly_capacity_1m_v
    WHERE year_num = 2026  
      AND station_code IS NOT NULL
    GROUP BY station_code, station_name, station_type, src_region_name, src_province_name
),
-- 3. 【核心修改1】存量/增量有效场站+【全年统一容量】，全年仅存量/增量各1条，彻底去重
valid_stations AS (
    -- 存量：全年1条，取【全年有效的存量场站】+ 存量装机容量（全年维度）+ 累计并网容量
    SELECT 
        2026 AS year_num,
        station_code,
        '存量' AS energy_type,
        MAX(stock_grid_capacity_mw) AS capacity_mw,  -- 存量全年统一容量（取最大值/也可SUM/AVG，按需改）
        MAX(cumulative_grid_capacity_mw) AS cumulative_grid_capacity_mw  -- 存量全年累计并网容量
    FROM ads.ads_stnops_energy_monthly_capacity_1m_v
    WHERE year_num = 2026
      AND station_code IS NOT NULL
      AND stock_grid_capacity_mw > 0
    GROUP BY station_code
    UNION ALL
    -- 增量：全年1条，取【全年有效的增量场站】+ 增量装机容量（全年维度）+ 累计并网容量
    SELECT 
        2026 AS year_num,
        station_code,
        '增量' AS energy_type,
        MAX(incr_grid_capacity_mw) AS capacity_mw,  -- 增量全年统一容量
        MAX(cumulative_grid_capacity_mw) AS cumulative_grid_capacity_mw  -- 增量全年累计并网容量
    FROM ads.ads_stnops_energy_monthly_capacity_1m_v
    WHERE year_num = 2026
      AND station_code IS NOT NULL
      AND incr_grid_capacity_mw > 0
    GROUP BY station_code
),
-- 4. 基础维度（整合场站、【全年统一容量】、年月，核心改：先关联场站类型，再笛卡尔积关联12个月）
base_dim AS (
    SELECT 
        ym.`year`, 
        ym.`month`, 
        vs.station_code, 
        vs.energy_type,
        vs.capacity_mw,  -- 直接取全年统一容量，不再按月取
        vs.cumulative_grid_capacity_mw,  -- 直接取全年统一累计容量
        -- 等效小时数从宽表按月份匹配
        CASE ym.`month`
            WHEN 1 THEN eqh.month_1 WHEN 2 THEN eqh.month_2 WHEN 3 THEN eqh.month_3 
            WHEN 4 THEN eqh.month_4 WHEN 5 THEN eqh.month_5 WHEN 6 THEN eqh.month_6
            WHEN 7 THEN eqh.month_7 WHEN 8 THEN eqh.month_8 WHEN 9 THEN eqh.month_9 
            WHEN 10 THEN eqh.month_10 WHEN 11 THEN eqh.month_11 WHEN 12 THEN eqh.month_12
        END AS month_should_hours,
        -- 月技改提升电量，空值默认0
        COALESCE(CASE ym.`month`
            WHEN 1 THEN te.month_1 WHEN 2 THEN te.month_2 WHEN 3 THEN te.month_3 
            WHEN 4 THEN te.month_4 WHEN 5 THEN te.month_5 WHEN 6 THEN te.month_6
            WHEN 7 THEN te.month_7 WHEN 8 THEN te.month_8 WHEN 9 THEN te.month_9 
            WHEN 10 THEN te.month_10 WHEN 11 THEN te.month_11 WHEN 12 THEN te.month_12
        END, 0) AS month_tech_energy,
        si.station_name,
        si.station_type,
        si.src_region_name,
        si.src_province_name
    FROM valid_stations vs
    INNER JOIN station_info si 
        ON vs.station_code = si.station_code
    CROSS JOIN year_month_2026 ym  -- 笛卡尔积：1个场站类型 关联 12个月，完美匹配需求
    -- 关联等效小时数宽表
    INNER JOIN dljy_fill.fill.fill_power_monthly_equiv_hours eqh
        ON vs.station_code = eqh.station_id 
        AND vs.year_num = eqh.`year`
    -- 关联技改提升电量宽表（2026年）
    LEFT JOIN dljy_fill.fill.fill_power_increase_event te
        ON vs.station_code = te.station_id 
        AND vs.year_num = te.`year`
),
-- 5. 新增参数：衰减率、系数、调整系数（表中无year字段，仅按场站编码关联）
year_coeff AS (
    SELECT
        station_id AS station_code,
        COALESCE(attenuation_rate, 0) AS attenuation_rate,  -- 衰减率（空值默认0）
        COALESCE(reporting_coefficient, 1) AS coeff,  -- 系数（空值默认1）
        COALESCE(adjustment_coefficient, 1) AS adjust_coeff  -- 调整系数（空值默认1）
    FROM dljy_fill.fill.fill_power_year_weight_coefficient
),
-- 6. 月度故障率（从fill_power_error_rate取2026年对应月份数据）
month_error_rate AS (
    SELECT
        fer.station_id AS station_code,
        ym.`month`,
        CASE ym.`month`
            WHEN 1 THEN fer.month_1 WHEN 2 THEN fer.month_2 WHEN 3 THEN fer.month_3 
            WHEN 4 THEN fer.month_4 WHEN 5 THEN fer.month_5 WHEN 6 THEN fer.month_6
            WHEN 7 THEN fer.month_7 WHEN 8 THEN fer.month_8 WHEN 9 THEN fer.month_9 
            WHEN 10 THEN fer.month_10 WHEN 11 THEN fer.month_11 WHEN 12 THEN fer.month_12
        END AS month_fault_rate
    FROM year_month_2026 ym
    CROSS JOIN dljy_fill.fill.fill_power_error_rate fer
    WHERE fer.`year` = 2026
),
-- 7. 月度预估限电率（沿用原逻辑，取2026年对应月份数据）
month_limit_rate AS (
    SELECT
        flr.station_id AS station_code,
        ym.`month`,
        CASE ym.`month`
            WHEN 1 THEN flr.month_1 WHEN 2 THEN flr.month_2 WHEN 3 THEN flr.month_3 
            WHEN 4 THEN flr.month_4 WHEN 5 THEN flr.month_5 WHEN 6 THEN flr.month_6
            WHEN 7 THEN flr.month_7 WHEN 8 THEN flr.month_8 WHEN 9 THEN flr.month_9 
            WHEN 10 THEN flr.month_10 WHEN 11 THEN flr.month_11 WHEN 12 THEN flr.month_12
        END AS month_limit_rate
    FROM year_month_2026 ym
    CROSS JOIN dljy_fill.fill.fill_power_limit_rate flr
    WHERE flr.`year` = 2026
),
-- 8. 计算月度应发电量、月度计划电量（逻辑不变，值按月度动态计算）
calc_month_plan AS (
    SELECT 
        bd.`year`,
        bd.`month`,
        bd.station_code,
        bd.energy_type,
        bd.station_name,
        bd.station_type,
        bd.src_region_name,
        bd.src_province_name,
        bd.capacity_mw,  -- 全年统一容量
        bd.cumulative_grid_capacity_mw,  -- 全年统一累计容量
        ROUND(
            (bd.month_should_hours * bd.capacity_mw /10 + bd.month_tech_energy * yc.coeff) 
            * (1 - yc.attenuation_rate),
            6
        ) AS month_should_energy,
        ROUND(
            (bd.month_should_hours * bd.capacity_mw/10 + bd.month_tech_energy * yc.coeff) 
            * (1 - yc.attenuation_rate)
            * (1 - COALESCE(mlr.month_limit_rate, 0) - COALESCE(mer.month_fault_rate, 0))
            * yc.adjust_coeff,
            6
        ) AS month_plan_energy
    FROM base_dim bd
    LEFT JOIN year_coeff yc
        ON bd.station_code = yc.station_code
    LEFT JOIN month_error_rate mer
        ON bd.station_code = mer.station_code AND bd.`month` = mer.`month`
    LEFT JOIN month_limit_rate mlr
        ON bd.station_code = mlr.station_code AND bd.`month` = mlr.`month`
),
-- 9. 宽表转换（核心：分组维度改为 年+场站+类型，完美实现全年存量1条、增量1条）
final_wide_table AS (
    SELECT 
        -- 核心改动2：拼接日期+递增序号作为版本号
        CONCAT(vs.base_version, '-', vs.seq) AS version,
        fwt.`year`,
        fwt.station_code,
        fwt.energy_type,
        fwt.station_name,
        fwt.station_type,
        fwt.src_region_name,
        fwt.src_province_name,
        fwt.capacity_mw,  -- 全年唯一容量值
        fwt.cumulative_grid_capacity_mw,  -- 全年唯一累计容量值
        MAX(CASE WHEN fwt.`month` = 1 THEN fwt.month_plan_energy ELSE 0 END) AS month_01,
        MAX(CASE WHEN fwt.`month` = 2 THEN fwt.month_plan_energy ELSE 0 END) AS month_02,
        MAX(CASE WHEN fwt.`month` = 3 THEN fwt.month_plan_energy ELSE 0 END) AS month_03,
        MAX(CASE WHEN fwt.`month` = 4 THEN fwt.month_plan_energy ELSE 0 END) AS month_04,
        MAX(CASE WHEN fwt.`month` = 5 THEN fwt.month_plan_energy ELSE 0 END) AS month_05,
        MAX(CASE WHEN fwt.`month` = 6 THEN fwt.month_plan_energy ELSE 0 END) AS month_06,
        MAX(CASE WHEN fwt.`month` = 7 THEN fwt.month_plan_energy ELSE 0 END) AS month_07,
        MAX(CASE WHEN fwt.`month` = 8 THEN fwt.month_plan_energy ELSE 0 END) AS month_08,
        MAX(CASE WHEN fwt.`month` = 9 THEN fwt.month_plan_energy ELSE 0 END) AS month_09,
        MAX(CASE WHEN fwt.`month` = 10 THEN fwt.month_plan_energy ELSE 0 END) AS month_10,
        MAX(CASE WHEN fwt.`month` = 11 THEN fwt.month_plan_energy ELSE 0 END) AS month_11,
        MAX(CASE WHEN fwt.`month` = 12 THEN fwt.month_plan_energy ELSE 0 END) AS month_12,
        CURRENT_TIMESTAMP AS create_time
    FROM calc_month_plan fwt
    -- 核心改动3：关联版本序号CTE
    CROSS JOIN version_seq vs
    GROUP BY 
        vs.base_version, vs.seq,  -- 新增分组字段（版本号相关）
        fwt.`year`,
        fwt.station_code,
        fwt.energy_type,
        fwt.station_name,
        fwt.station_type,
        fwt.src_region_name,
        fwt.src_province_name,
        fwt.capacity_mw,
        fwt.cumulative_grid_capacity_mw
    HAVING 
        (month_01 + month_02 + month_03 + month_04 + month_05 + month_06 + 
         month_07 + month_08 + month_09 + month_10 + month_11 + month_12) > 0
)
-- 主查询：输出2026年度发电量计划宽表，【最终结果：每个场站 存量1行 + 增量1行】
SELECT * FROM final_wide_table
order by version,energy_type desc,station_type desc,src_region_name,src_province_name,station_code
LIMIT 1000;